# Data Science Project 1 -  Improve the mass accuracy of spectra measured by Orbitrap mass spectrometers (orbitrap)

**Client:** Atmospheric Physical Chemistry group, INAR, University of Helsinki

**Description:** Motivation: When using a mass spectrometer, the measured mass usually shifts from its true mass. Hence, a mass calibration is an important procedure before allocating chemical formulae to the measured masses. A good mass calibration may greatly reduce the efforts of further analysis and increase the reliability of the results. Goals: Improve the mass calibration procedure for Orbitrap raw data, and perhaps for data measured by other mass spectrometers, e.g, TOF-MS Main tasks: 1) Test several fitting function for mass correction, and recommend one or a few that works best. 2) Test the performance of different parameters for mass correction, e.g., number of mass of the species for calibration.

**Data and tools:** Data: raw spectrum data measured by Orbitrap mass spectrometer. Tools: a) Orbitool, provided by the client. Orbitool will be used for reading the raw data and remove the noise, i.e., prepare the data for this analysis; b) Any programming language, which will be used to investigate this mass calibration problem.

## Environment

The environment should have the dependencies to run Orbitool and this notebook.

```bash
# Import environment
conda env create -f environment.yml

# Import kernel to jupyter
ipython kernel install --user --name=orbitool

# Export the environment to file
conda env export --no-builds > environment.yml
```

## Notebook practices

* Clean outputs before commiting!

## Dependencies

In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import seaborn as sns
import matplotlib.pyplot as plt
import random
from scipy.signal import find_peaks
from scipy.optimize import leastsq
from pyteomics import mass
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
import random

## Data exploration

### Read the spectrum data

* Put the directory of spectrum CSV files to ```spectrum_data_directory``` variable.
    * If the begining of the CSV files contains some time information, then set ```contains_time_data``` variable to ```True```.
* Put the path of the peak list file to ```peak_list_file``` variable.
* If you just want to test the code, then you can speed up the code by setting the size of spectrum sample to ```random_sample_size``` variable.

In [ ]:
spectraRoot = ""
peaklistRoot = ""
spectrum_data_directory = spectraRoot + "data/First Large Deviation file-20210415T191813Z-001/First Large Deviation file/2 mins"
contains_time_data = False #Spectrum files contains time data in forst 3 rows?
peak_list_file = peaklistRoot + "peak list/peaklist_1e5_background.csv"
random_sample_size = 5 #Take sample spectrums to speed up things. If 'None' then uses all the spectrums

spectrum_data_files = []
time_data = []
spectrums = []

# Get data files
for file in [f for f in listdir(spectrum_data_directory) if isfile(join(spectrum_data_directory, f))]:
    path_to_file = join(spectrum_data_directory, file)
    #print(path_to_file)
    if file.endswith(".csv"):
        spectrum_data_files.append(path_to_file)

# Take sample spectrums to speed up things
if random_sample_size:
    spectrum_data_files = random.sample(spectrum_data_files, min(random_sample_size, len(spectrum_data_files)))

# Read the spectrum files
for file in spectrum_data_files:
    if contains_time_data:
        time_data.append(pd.read_csv(file)[:2])
        spectrums.append(pd.read_csv(file, skiprows = 3).sort_values(by=['mz']))
    else:
        spectrums.append(pd.read_csv(file))

peak_list = dict(pd.read_csv(peak_list_file).values)

# Lookup
spectrums[0].head(5)

### Plot  the spectrums

* Plot all the spectrums with actual ions (from peak list file).

In [ ]:
%matplotlib notebook
# seaborn version should be at least 0.11!
print(sns.__version__)

# Concat files to same data frame with 'spectrum' separator column
tmp = pd.concat(spectrums, keys=range(len(spectrum_data_files)), names=["spectrum"])

# Plot the spectrums
sns.set_theme(style="whitegrid")
fig, ax = plt.subplots(figsize=(8,6))
sns.lineplot(data=tmp, ax=ax, x="mz", y="intensity", hue="spectrum", palette="tab10")
ax.legend([],[], frameon=False)
ax.ticklabel_format(useOffset=False)

# Plot the actual masses. Line height is taken from max intensity with +-eps interval.
eps = 0.0001
for formula, mz in peak_list.items():
    max_intensity = max([x for x in tmp.loc[(tmp["mz"] > mz - eps) & (tmp["mz"] < mz + eps)].intensity.values] + [0.0])
    plt.plot((mz,mz), (0.0,max_intensity), linestyle="--")

## Detect peaks

* Detect peaks
* Calculate/get some information about the peaks

In [ ]:
def find_peak_indices(data):
    '''
    Logic is same as in Koli's implementation. Tested that produces same results with spectrums in
    "data/CI-orbi_20201117165601 folder (first smal deviation file)/1 min" folder.
    '''
    peak_start_indices = [0] #Initialize with first start index
    peak_end_indices = []
    for i, row in data.iterrows():
        if i == 0:
            continue;
        if data.iloc[i]["intensity"] == 0.0 and data.iloc[i - 1]["intensity"] == 0.0:
            peak_start_indices.append(i)
            peak_end_indices.append(i - 1)
    peak_end_indices.append(len(data) - 1) #Finalize with last end index
    return np.column_stack((peak_start_indices, peak_end_indices))

peak_informations = []
for spectrum in spectrums:
    peak_information = []
    for start, end in find_peak_indices(spectrum):
        data = spectrum.iloc[start:(end+1)]
        max_intensity = data["intensity"].max()
        width = data["mz"].max() - data["mz"].min()
        average_mz = np.average(data["mz"], weights=data["intensity"])
        peak_information.append({
            "start": start,
            "end": end,
            "max_intensity": max_intensity,
            "average_mz": average_mz,
            "width": width
        })
    peak_informations.append(pd.DataFrame(peak_information))
    
peak_informations[0].head(5)

### Handle 'double peaks'

* Converts 'double peaks' to separate peaks.

In [ ]:
# TODO

## Fit Gaussian curve to peaks

* Fit Gaussian curve to peaks and get fitted means.
* Set the wanted resolution to ```default_resolution``` variable.

In [ ]:
default_resolution = 280000

# x: data; a: height; x0: position; c: sigma or width
gauss  = lambda x, a, mu, sigma: a*np.exp(-(x-mu)**2/(2*sigma**2))

def fit_gaussian(peak, resolution=default_resolution, show=False, use_rolling = None):
    '''
    returns: a: 'height'; mu: 'position'; sigma: 'width'
    '''
    intensity = peak["intensity"] if not use_rolling else peak["intensity"].rolling(use_rolling, center=True, min_periods=1).mean()
    mu = np.average(peak["mz"], weights=intensity)
    sigma = mu/(resolution*2*np.sqrt(2*np.log(2)))
    errfunc  = lambda p, x, y: (y - gauss(x, p[0], p[1], sigma))
    init  = [intensity.max(), mu]
    out = leastsq(errfunc, init, args=(peak["mz"], intensity), maxfev = 200)
    c = out[0]
    if show:
        x = np.linspace(peak["mz"].min(), peak["mz"].max(), 1000)
        fig, ax = plt.subplots(figsize=(8,6))
        ax.ticklabel_format(useOffset=False)
        plt.plot(peak["mz"], peak["intensity"], "b")
        if use_rolling:
            plt.plot(peak["mz"], intensity, "b", linestyle="--")
        plt.plot(x, gauss(x, c[0], c[1], sigma), "g")
        plt.plot((c[1],c[1]), (0.0,c[0]), "g", linestyle="--")
    return c[0], c[1], sigma

# Test Fitting
n_spectrum = 0
n_peak = 163

spectrum = spectrums[n_spectrum]
peak_information = peak_informations[n_spectrum]
peak = peak_information.iloc[n_peak]
data = spectrum.iloc[int(peak["start"]):int(peak["end"] + 1)]
a, mu, sigma = fit_gaussian(data, show=True, use_rolling = 6)

In [ ]:
# Add 'observed_mz' column
for spectrum, peak_information in zip(spectrums, peak_informations):
    fitted_means = []
    for i, peak in peak_information.iterrows():
        data = spectrum.iloc[int(peak["start"]):int(peak["end"] + 1)]
        a, mu, sigma = fit_gaussian(data)
        fitted_means.append(mu)
    peak_information["observed_mz"] = fitted_means
    
peak_informations[0].head(5)

## Classify ions

* Associate every ion from the ```peak_list``` to closest detected peak if there is detected peaks closer than ```theta```.

In [ ]:
def identify_peaks(peak_information, peak_list, theta = 0.01):
    peak_information["formula"] = None # For strings
    peak_information["formula_mz"] = np.nan # For floats
    for key in peak_list.keys():
        formula = key
        true_mz = peak_list[key]
        closest = peak_information[np.abs(peak_information["observed_mz"] - true_mz) < theta]
        if len(closest) > 0:
            closest = closest.iloc[(closest['observed_mz'] - true_mz).abs().argsort()[:1]]
            #print("True: {}; Observed: {}; Index: {}".format(true_mz, closest["observed"].values[0], closest.index.values[0]))
            peak_information.at[closest.index.values[0], "formula"] = formula
            peak_information.at[closest.index.values[0], "formula_mz"] = true_mz
    peak_information["error"] =  peak_information["formula_mz"] - peak_information["observed_mz"]

# Identify peaks
for peak_information in peak_informations:
    identify_peaks(peak_information, peak_list)

# Lookup
peak_informations[0].loc[peak_informations[0]["formula"].notnull()]

## Uncertainty of mass

* Analyse uncertainty of mass.

The calculateMassUncertainty function takes a dataframe containing a processed spectrum with the following columns: formula, mz, observerd.
It returns a dictionary with the uncertainty for each element calculated as the average of all the uncertainties for wich the element was present.
By default it will take perform a weighted averged based on the number of elements in the compounds/ions.
If weighted is set to False, it will only average the uncertainty based solely on the presence of the element.


In [ ]:
%matplotlib notebook

def calculateMassUncertainty(peak_information, weighted=True, dfOutput=True, show=False):
    elements = {}
    for index, row in peak_information.iterrows():
        ion = row["formula"]
        if not ion:
            continue;
        ion = ion if ion[-1] != '-' else ion[:-1]
        tmp = mass.Composition(formula=ion)
        v = row["error"]
        total = sum(tmp.values())
        for e in tmp.keys():
            f = 1
            if weighted:
                f = tmp[e] / total
            if e not in elements:
                elements[e] = [v*f]
            else:
                elements[e].append(v*f)
    for e in elements.keys():
        elements[e] = sum(elements[e]) / len(elements[e])
    if show:
        keys = elements.keys()
        values = elements.values()
        plt.figure(1)
        plt.bar(keys, values)
    if dfOutput:
        df = pd.DataFrame(elements.items(), columns=['Element', 'Uncertainty'])
        return df
    else:
        return elements
    
tmp = pd.concat(peak_informations, keys=range(len(spectrum_data_files)), names=["spectrum"])
calculateMassUncertainty(tmp, True, True,True)

## Fit linear/polynomial regression model

* Fits linear/polynomial model to ```error``` column.
* ```MultiPolyModel``` is collection of polynomial models with different degrees (can also be used only with single degree). Prediction result is averaged result from the models (can be weighted).

### Useful links

* https://realpython.com/linear-regression-in-python/
* https://towardsdatascience.com/polynomial-regression-which-python-package-to-use-78a09b0ac87b
* https://stackoverflow.com/questions/34373606/scikit-learn-coefficients-polynomialfeatures

In [ ]:
%matplotlib notebook

class MultiPolyModel:
    '''
    Fits multiple polynomial models with different degrees. Prediction value is average value of 
    predicted values from different models (weigths can be used).
    
    If 'max_degree = min_degree' then this is just simple single polynomial model with degree of 
    'max_degree' (or 'max_degree').
    '''
    degrees = None
    weights = None
    models = None
    
    debug = False
    
    @classmethod
    def __init__(self, degrees, weights = None, debug = False):
        '''
        Initializes model.
        
        Arguments:
            min_degree: Minium degree of model (first model).
            max_degree: Maximum degree of model (last model).
            weight: None, Integer, or List
                None: Weights are degrees in inversed order (exmpl. degrees: [1,2,3] then weights: [3,2,1]).
                Integer: All models have same weight (value doesn't matter).
                List: Weights (size should be amout of models i.e. 'max_degree-min_degree+1').
        '''
        self.models = []
        self.degrees = np.array(degrees)
        self.debug = debug
        
        if weights and type(weights) == int:
            self.weights = np.repeat(weights, len(self.degrees))
        elif weights and type(weights) == list:
            self.weights = np.array(weights)
        else:
            self.weights = 1 / self.degrees
            
        if self.debug:
            print("Degrees: {}; Weights: {}".format(self.degrees, self.weights))
    
    @classmethod
    def fit(self, x, y, show=False, color="r", ax = None):
        for degree in self.degrees:
            self.models.append(fit_poly_reg_model(x, y, degree = degree))
        if show:
            x_plot = np.linspace(min(x), max(x), 1000)
            preds = self.predict(x_plot)
            sns.set_theme(style="whitegrid")
            if not ax:
                fig, ax = plt.subplots(figsize=(8,6))
            sns.scatterplot(x=x, y=y, ax=ax, color=color)
            plt.plot(x_plot, preds, linestyle="-", color=color)
    
    @classmethod
    def predict(self, x):
        results = []
        for model in self.models:
            results.append(model.predict(x.reshape(-1, 1)))
        return np.average(np.array(results), axis=0, weights = self.weights)
    
    @staticmethod
    def fit_poly_reg_model(x, y, degree, show = False, ax = None, color="r"):
        poly_features = PolynomialFeatures(degree)
        model = make_pipeline(poly_features, LinearRegression()).fit(x.reshape(-1, 1), y)
        if show:
            print("Coefficients: {}".format(model.steps[1][1].coef_[1:]), "Independent: {}".format(model.steps[1][1].intercept_))
            preds = model.predict(x.reshape(-1, 1))
            sns.set_theme(style="whitegrid")
            if not ax:
                fig, ax = plt.subplots(figsize=(8,6))
            sns.scatterplot(x=x, y=y, ax=ax, color=color)
            plt.plot(x, preds, linestyle="-", color=color)
        return model
        
    

peak_information = peak_informations[1]
peak_information = peak_information[peak_information["error"].notnull()]

# TODO: better peak selection
train_data = peak_information[peak_information["max_intensity"] > 10000]

fig, ax = plt.subplots(figsize=(8,6))
sns.scatterplot(x=peak_information["observed_mz"].values, y=peak_information["error"].values*-1, ax=ax)
model = MultiPolyModel([1,2,3,4,5], weights = 1, debug = True)
model.fit(train_data["observed_mz"].values, train_data["error"].values*-1, show = True, ax=ax)

## Peak selection methods

### Method 1: Select best n peaks

This first idea is to select a set amount n of the best peaks to lead the calibration process. It is possible to set a treshold to specifiy the minimal value of error accepted.  
The threshold criteria is secondary to the number of peaks, meaning that if the threshold isn't met and the number of peaks selected is under n, it will put those below the threshold to reach n. To change this set the forceThreshold parameter to True.  
To disable thr threshold set it to 0 (default value).  
The treshold is treated as absolute value, so negative values don't matter.  
This method provides the remaining peaks as a test set, and the size can be specified with the test_n parameter. If left at 0 it will return all the discarded data for testing.  
The randomTestSamples parameter allows to sample randomly from the remaining data, otherwise they're provided ordered by least uncerteinty value.

In [ ]:
def selectBestPeaks(data, n=10, threshold=0, forceThreshold=False, test_n=0, randomTestSamples=True, debug=False):
    if n <=0:
        return None
    result = data.copy()
    result["absolute_error"] =  abs(data["error"])
    result = result.sort_values(by=['absolute_error'], ascending=True) #specified ascending parameter for flexibility
    diff = result
    test_n = abs(test_n)
    
    if debug:    
        print(result)
    
    if threshold:
        threshold = abs(threshold)
        filtered = result[result['absolute_error']<=threshold]
    
    if len(result) > n:
        if threshold:
            if len(filtered) > n:
                result = filtered[0:n]
            else:
                if forceThreshold:
                    result = filtered
                else:
                    result = result[0:n]
        else:
            result = result[0:n]
    else:
        if forceThreshold and threshold:
            result = filtered
        else:
            x = max(len(result), n)
            result = result[0:x]
            
    diff = pd.concat([result,diff]).drop_duplicates(keep=False)
    if test_n != 0:
        test_n = min(test_n, len(diff))
        if randomTestSamples:
            diff = diff.sample(test_n)
        else:
            diff = diff[0:test_n]
    return result, diff

a, b = selectBestPeaks(peak_informations[0], 5, 1.397722e-06, False, 10, True)

## Test first peak selection method

In [ ]:
random.seed(1)

train_information = []
# Single degree models
for n in [10, 20, 30, 40, 50, 100]:
    for max_degree in range(1, 11):
        for single_degree in [True, False]:
            for weight in [1, None]:
                min_degree = 1
                if single_degree:
                    min_degree = max_degree
                mses = []
                for peak_information in peak_informations:
                    peak_information = peak_information[peak_information["error"].notnull()]
                    
                    train, test = selectBestPeaks(peak_information, n=n, test_n=50)
                    model = MultiPolyModel(list(range(min_degree, max_degree + 1)), weights = weight)
                    model.fit(train["observed_mz"].values, train["error"].values*-1)
                    preds = model.predict(test["observed_mz"].values)
                    mse = mean_squared_error(test["error"].values*-1, preds)
                    mses.append(mse)
                train_information.append({
                    "min_degree": min_degree,
                    "max_degree": max_degree,
                    "weight": weight,
                    "n": n,
                    "mean_mse": np.mean(mses),
                    "std_mse": np.std(mses),
                    "min_mse": np.min(mses),
                    "max_mse": np.max(mses)
                })
        
pd.DataFrame(train_information).sort_values(by=['mean_mse', "std_mse", "max_mse"], ascending=True).head(20)